In [1]:
# from belle2_enhanced_framework import create_belle2_framework
# framework = create_belle2_framework(particle_type='vpho', memory_budget_gb=32.0,energy_condition='5S_scan')
# processes = framework.load_particle_data(
#     "/gpfs/group/belle2/users2022/kyldem/photoneff_updated/parquet_storage/try5",
#     particle='vpho'  # Automatically uses VPHO_KEYS columns
# )

In [2]:
# !pip3 install polars
# !pip install dill
from belle2_enhanced_framework import create_belle2_framework, ParticleDataLoaderV3
framework = create_belle2_framework(energy_condition='4S_offres')
loader = ParticleDataLoaderV3(particle_type='vpho', config=framework.config)
processes = loader.load_flat_structure(
    base_dir='/pnfs/desy.de/belle/local/user/sraiz/photon_efficiency/gammaEffKLM_Prompt_combinedFilesMerged',
    data_type='selected'  # or 'matched'
    
)

✅ C++ histogram acceleration: ACTIVE
📊 System analysis:
   Available memory: 495.8 GB
   Optimal chunk size: 1,000,000 elements
📊 Process limits: soft=2060546, hard=2060546
🔧 Max Python threads: 16
✅ C++ histogram acceleration available

        ╔════════════════════════════════════════════════════╗
        ║          Belle II Layer 2 Framework                ║
        ║                                                    ║
        ║  Memory Budget:  16.0 GB                     ║
        ║  C++ Acceleration:  Enabled                 ║
        ║  Cache Directory: /afs/desy.de/user/k/kyldem/.belle2_cache║
        ╚════════════════════════════════════════════════════╝
        

        ╔════════════════════════════════════════════════════╗
        ║     Belle II Enhanced Production Framework v3      ║
        ║                                                    ║
        ║  ✓ Clean weight/data separation                   ║
        ║  ✓ Unified histogram pipeline                     ║
    

OSError: [Errno 121] Remote I/O error: '/pnfs/desy.de/belle/local/user/sraiz/photon_efficiency/gammaEffKLM_Prompt_combinedFilesMerged'

In [ ]:
from belle2_enhanced_framework import create_belle2_framework,ParticleDataLoaderV3
from streamlined_belle2_generator import QuickBelle2Generator
from belle2_data_validation import DataValidationFramework

# Step 1: Generate validated test data
# generator = QuickBelle2Generator('./synthetic_test_data')
# files = generator.generate_standard_files(scale=10)  # Start small

# Step 2: Validate structural integrity
# validator = DataValidationFramework()
# for filepath in files.values():
#     report = validator.validate_dataset(filepath)
#     assert report['overall_score'] > 90, f"Validation failed for {filepath}"

# Step 3: Load through standard framework
framework = create_belle2_framework(energy_condition='4S_offres')
loader = ParticleDataLoaderV3(particle_type='vpho', config=framework.config)
processes = loader.load_flat_structure(
    base_dir='./synthetic_test_data',
    data_type='selected'
)

# Step 4: Execute analysis pipeline
results = framework.run_progressive_analysis(
    processes,
    variable='pRecoil',
    bins=100,
    range=(0.1, 6.0),
    stages=['baseline', 'candidates']
    
)


    ═══════════════════════════════════════════════════════════════
                        Belle II Layer 2
                   Compute-First Data Structures
    ═══════════════════════════════════════════════════════════════
    
    Components:
    ├── UnifiedLazyDataFrame
    │   └── Manifests compute graphs as DataFrames
    ├── OptimizedUltraLazyDict
    │   └── Process-aware container with groups
    ├── MaterializationController
    │   └── Intelligent format selection
    ├── GraphOptimizationEngine
    │   └── Automatic compute graph optimization
    └── MemoryAwareExecutor
        └── Adaptive execution with spilling
    
    Key Features:
    • Zero-copy operations through compute graphs
    • Automatic billion-row handling
    • C++ acceleration for critical operations
    • Physics-specific optimizations
    • Full transformation tracking
    
    Usage:
    >>> framework = Belle2Layer2Framework(memory_budget_gb=16.0)
    >>> data = framework.load_processes("/data/belle2/

/afs/desy.de/user/k/kyldem/belle2_framework_code/layer2_unified_lazy_dataframe.py:424: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  if hasattr(frame, 'schema'):


✅ C++ histogram acceleration: ACTIVE
📊 System analysis:
   Available memory: 492.4 GB
   Optimal chunk size: 1,000,000 elements
📊 Process limits: soft=2060546, hard=2060546
🔧 Max Python threads: 16
      ✅ mumu-offprompt: Loaded with weight=0.2629

   Loading ee-offprompt...
      ✅ ee-offprompt: Loaded with weight=10.5153

   Loading taupair-offprompt...
      ✅ taupair-offprompt: Loaded with weight=0.2629

   Loading uubar-offprompt...
      ✅ uubar-offprompt: Loaded with weight=0.2629

   Loading ddbar-offprompt...
      ✅ ddbar-offprompt: Loaded with weight=0.2472

   Loading ssbar-offprompt...
      ✅ ssbar-offprompt: Loaded with weight=0.2472

   Loading ccbar-offprompt...
      ✅ ccbar-offprompt: Loaded with weight=0.2629

   Loading data-offprompt...
      ✅ data-offprompt: Loaded with weight=1.0000
✅ Successfully loaded 8 processes

🎯 PROGRESSIVE ANALYSIS: pRecoil

📊 STAGE: baseline
------------------------------
📊 Histogram computation for 'pRecoil' using CPP_ACCELERATED stra

In [1]:
!pip install psutil
from importlib import reload
import belle2_enhanced_framework
reload(belle2_enhanced_framework)
from belle2_enhanced_framework import create_belle2_framework, OptimizedUltraLazyDict
# Run analysis for each variable
# Define variables with their binning
variables = [
    ('pRecoil', 100, (0.1, 6)),

    # ('mu1P', 60, (0, 3)),
    # ('mu2P', 60, (0, 3)),
]
# Define physics cuts
full_cut = 'mu1nCDCHits>4&mu2nCDCHits>4&0.8>mu1clusterEoP&0.8>mu2clusterEoP&2.6179938779914944>pRecoilTheta>0.29670597283903605&11>totalMuonMomentum&absdPhi>1.5707963267948966&2.03>mu1Theta>0.61&2.03>mu2Theta>0.61&(absdPhiMu1>0.4014257279586958|absdThetaMu1>0.4014257279586958)&(absdPhiMu2>0.4014257279586958|absdThetaMu2>0.4014257279586958)&0.35>mu1clusterE&0.35>mu2clusterE&3>abs(m2Recoil)&min_deltaMuPRecoil>-0.01'
user_cuts = full_cut.split('&') if full_cut else []
all_results = {}
for var_name, bins, range_tuple in variables:
    print(f"\n📊 Analyzing {var_name}...")
    results = framework.run_progressive_analysis(
        processes,
        variable=var_name,
        bins=bins,
        range=range_tuple,
        cuts=user_cuts,
        stages=['baseline', 'candidates', 'cuts'],  # Standard progression
    )
    all_results[var_name] = results

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: Loading egg at /cvmfs/belle.cern.ch/el9/externals/v02-03-00/Linux_x86_64/common/lib/python3.11/site-packages/PyFastBDT-5.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

    ═══════════════════════════════════════════════════════════════
                        Belle II Layer 2
                   Compute-First Data Structures
    ═══════════════════════════════════════════════════════════════
    
    Components:
    ├── UnifiedLazyDataFrame
    │   └── Manifests compute graphs as DataFrames
    ├── OptimizedUltraLazyDict
    │   └── Process-aware container with groups
    ├── MaterializationController
    │   └── Intelligent format sel

NameError: name 'framework' is not defined

In [3]:
results

NameError: name 'results' is not defined